In [21]:
import datetime
lipidname = "PBSM"
tail = "TCC CCCCC" 
link = "A A"
head = "C P"

description = ";   A general model sphingomyelin (SM) lipid \n;      C16:0 palmitic acid , C20:0 arachidoyl   \n"
modeledOn=";   This topology follows the standard Martini 2.0 lipid definitions and building block rules.\n; Reference(s): \n;   S.J. Marrink, A.H. de Vries, A.E. Mark. Coarse grained model for semi-quantitative lipid simulations. JPC-B, 108:750-760, \n;   2004. doi:10.1021/jp036508g \n;   S.J. Marrink, H.J. Risselada, S. Yefimov, D.P. Tieleman, A.H. de Vries. The MARTINI force field: coarse grained model for \n;   biomolecular simulations. JPC-B, 111:7812-7824, 2007. doi:10.1021/jp071097f \n;   T.A. Wassenaar, H.I. Ingolfsson, R.A. Bockmann, D.P. Tieleman, S.J. Marrink. Computational lipidomics with insane: a versatile \n;   tool for generating custom membranes for molecular simulations. JCTC, 150410125128004, 2015. doi:10.1021/acs.jctc.5b00209\n; Created: "

now = datetime.datetime.now()
membrane="testmembrane"
insane="../insane+SF.py"
mdparams="../test.mdp"
martinipath="../martini.ff/"
ITPCatalogue="./epithelial.cat"
ITPMasterFile="martini_v2_epithelial.itp"

modeledOn+= now.strftime("%Y.%m.%d")+"\n"

# Cleaning up intermediate files from previous runs
!rm -f *#*
!rm -f *step*
!rm -f {membrane}*

In [22]:
import fileinput
import os.path

print("Create itp")
!python {martinipath}/lipid-martini-itp-v06.py -o {lipidname}.itp -alname {lipidname} -name {lipidname} -alhead '{head}' -allink '{link}' -altail '{tail}'

#update description and parameters
with fileinput.FileInput(lipidname+".itp", inplace=True) as file:
    for line in file:
        if line == ";   This is a ...\n":
            print(description, end='')
        elif line == ";   Was modeled on ...\n":
            print(modeledOn, end='')
        else:
            print(line, end='')



Create itp
The Martini lipid itp generator version 0.6  Args are: -o PBSM.itp -alname PBSM -alhead 'C P' -allink 'A A' -altail 'TCC CCCCC'


In [23]:
#Add this ITP file to the catalogue file
if not os.path.exists(ITPCatalogue):
    ITPCatalogueData = []
else:
    with open(ITPCatalogue, 'r') as file :
        ITPCatalogueData = file.read().splitlines()

    ITPCatalogueData = [x for x in ITPCatalogueData if not x==lipidname+".itp"]

ITPCatalogueData.append(lipidname+".itp")
    
with open(ITPCatalogue, 'w') as file :
    file.writelines("%s\n" % item for item in ITPCatalogueData)
    
#build ITPFile    
with open(martinipath+ITPMasterFile, 'w') as masterfile:
    for ITPfilename in ITPCatalogueData:
        with open(ITPfilename, 'r') as ITPfile :
            for line in ITPfile:
                masterfile.write(line)
    
print("Done")

Done


In [24]:
# build a simple membrane to visualize this species
!python2 {insane} -o {membrane}.gro -p {membrane}.top -d 0 -x 3 -y 3 -z 3 -sol PW -center -charge 0 -orient -u {lipidname}:1 -l {lipidname}:1 -itpPath {martinipath}

; X: 3.000 (4 lipids) Y: 3.000 (4 lipids)
; 16 lipids in upper leaflet, 16 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 384
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 385 0
; NDX System 1 384


In [25]:
import os #Operating system specific commands
import re #Regular expression library

print("Test")
print("Grompp")
grompp = !gmx grompp -f {mdparams} -c {membrane}.gro -p {membrane}.top -o  {membrane}.tpr
success=True
for line in grompp:
    if re.search("ERROR", line):
        success=False
    if re.search("Fatal error", line):
        success=False
    #if not success:
    print(line)
    
if success:
    print("Run")
    !export GMX_MAXCONSTRWARN=-1
    !export GMX_SUPPRESS_DUMP=1
    run = !gmx mdrun -v -deffnm {membrane}
    summary=""
    logfile = membrane+".log"
    if not os.path.exists(logfile):
        print("no log file")
        print("== === ====")
        for line in run:
            print(line)
    else:
        try:
            file = open(logfile, "r")
            fe = False
            for line in file:
                if fe:
                    success=False
                    summary=line
                elif re.search("^Steepest Descents.*converge", line):
                    success=True
                    summary=line
                    break
                elif re.search("Fatal error", line):
                    fe = True
        except IOError as exc:
            sucess=False;
            summary=exc;
    if success:
        print("Success")
    else:
        print(summary)

Test
Grompp
                      :-) GROMACS - gmx grompp, 2018.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2017, Th